In [6]:
import os

In [7]:
# Constants
OMEGA_BARYON = 0.046026

# file parts here
param_part = '../../param/parts/param.part'
doit_part = '../../param/parts/doit.part'

hydro_part = '../../param/parts/hydro.part'
SIDM_part = '../../param/parts/SIDM.part'

In [8]:
# Simulation Parameters
# Change these

N_part = 256
boxsize = 3 # in Mpc
code_directory = 1
mem_size = 8 # in GB

DM_type = '2cDM'
hydro_type = 'DM'
cross_section = 'power00_sigma1'
extra_info = 'var_2'

In [10]:
with open(param_part, 'r') as f:
    param_text = f.read()

with open(doit_part, 'r') as f:
    doit_text = f.read()

job_string = DM_type + f'_L{boxsize}N{N_part}_' + hydro_type + '_' + cross_section + '_' + extra_info
boxsize_kpc = boxsize * 1000
softening_length = boxsize_kpc / (29 * N_part )
mem_size_MB = mem_size * 1000 

# replacing text in parameter file
param_text = param_text.replace('{$MEMSIZE}', str(mem_size_MB))
param_text = param_text.replace('{$CODE_DIR}', str(code_directory))
param_text = param_text.replace('{$BOXSIZE}', str(boxsize_kpc))

param_text = param_text.replace('{$SOFTENING_LENGTH}', str(softening_length))
param_text = param_text.replace('{$SOFTENING_LENGTH_HALF}', str(softening_length / 2)) 

if DM_type != 'CDM':
    with open(SIDM_part, 'r') as f:
        param_text += f.read()

if hydro_type == 'HY':
    hydro_flag = 1
    omega_baryon = OMEGA_BARYON
    with open(hydro_part, 'r') as f:
        param_text += f.read()
else:
    hydro_flag = 0
    omega_baryon = 0
param_text = param_text.replace('{$OMEGA_BARYON}', str(omega_baryon))
param_text = param_text.replace('{$HY_FLAG}', str(hydro_flag))

# replacing text in doit script
doit_text = doit_text.replace('{__MEMSIZE__}', f'{mem_size}g')
doit_text = doit_text.replace('{__JOBSTRING__}', job_string)
doit_text = doit_text.replace('{__CODE_DIR__}', str(code_directory))

In [ ]:
print(param_text)

InitCondFile                                      /home/r408l055/projects/Medvedev/IC/IC-L3N2561
OutputDir                                         /home/r408l055/scratch/output/live1
SnapshotFileBase                                  snap
OutputListFilename                                ../TNGfiles/ExpansionList_L3N256
ICFormat                                          3
SnapFormat                                        3
TimeLimitCPU                                      21600 %6hour
CpuTimeBetRestartFile                             14400 %4hour
ResubmitOn                                        1
ResubmitCommand                                   /home/r408l055/projects/SIDM/code/doit-restart.sh
MaxMemSize                                        8000
TimeBegin                                         0.01
TimeMax                                           1
ComovingIntegrationOn                             1
PeriodicBoundariesOn                              1

CoolingOn                     

In [11]:
print(doit_text)

#!/bin/bash
#SBATCH --nodes=8
#SBATCH --ntasks-per-node=32
#SBATCH --mem-per-cpu=8g
#SBATCH --time=6:00:00
##SBATCH --mail-user=rtlow@ku.edu
##SBATCH --mail-type=ALL
#SBATCH --job-name=2cDM_L3N256_DM_power00_sigma1_var_2
#SBATCH --partition=sixhour
#SBATCH --constraint "intel"
#SBATCH --constraint=ib

# Which code directory
i=1
# Is this a restart run?
RESTART=0
# Point to output directory
OUTDIR="/home/r408l055/scratch/output"
# Point to final snapshot
FINAL_SNAP="${OUTDIR}/live${i}/snap_007.hdf5"


# Modification is rarely needed
RESTART_DIR="${OUTDIR}/live${i}/restartfiles/"
JOBNAME=$SLURM_JOB_NAME
PARAM='{__PARAM_FILE__}'
PARAM_PATH="../RUNS/boxes/${PARAM}"
LOG_PATH=/home/r408l055/scratch/logs/LOG_${JOBNAME}_$(date +"%Y_%m_%d_%H_%M_%S")

# Flags for iteration go/no-go
RESTART_EXISTS=1
N_FINAL_SNAP_EXISTS=0
N_RUN_TERMINATED=0
N_EXISTING_RUNS=0

GO=0
# Actual run starting code
~/remove_core.sh

module load hdf5/1.10.5
module load openmpi/4.0
mpiexec ./Arepo $PARAM_PATH $RESTART > $LO